In [1]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [2]:
import pandas as pd
import re

df = pd.read_csv("호주_세관_v2(new20).csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i].lower()+""+df_text[i])

df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)

In [3]:
df_add = df_add[:2]


In [4]:
#전처리
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

doc_list = []
for doc in df_add:

    #구두점 제거
    doc1 = "".join([i for i in doc if i not in string.punctuation]).strip()

    #숫자 제거
    doc2 = "".join([i for i in doc1 if not i.isdigit()])
    
    #월 제거
    month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
            'september', 'october', 'november', 'december', 'jan', 'feb', 'mar', 'apr',
             'may', 'jun','jul', 'aug', 'sep', 'oct', 'nov', 'dec']   

    doc3 = " ".join([i for i in doc2.split() if i not in month])
    doc_list.append(doc3)

In [5]:
#전처리한 문서에서 바이그램 추출
n_gram_range = (2, 2)
stop_words = "english"

candidate_list = []
for doc in doc_list:
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
    candidate = count.get_feature_names_out()
    candidate_list.append(candidate)

In [12]:
#문서 전체와 문서에서 추출한 키워드 수치화
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
bigram_keywords = []
top_n = 20

for i in range(len(doc_list)):
    doc_embeddings = model.encode([doc_list[i]])
    candidate_embeddings = model.encode(candidate_list[i])
    distances = cosine_similarity(doc_embeddings, candidate_embeddings)
    bigram_keywords.append([candidate_list[i][index] for index in distances.argsort()[0][-top_n:]])

In [13]:
#문서와 가장 유사한 키워드 벡터화
bigram_embeddings = []
for i in range(len(doc_list)):
    bigram_embedding = []
    for keyword in bigram_keywords[i]:
        bigram_embedding.append(model.encode(keyword))
    bigram_embeddings.append(bigram_embedding)

In [14]:
#keyword : 불러온 호주 키워드 중 번역 Column 추출
keyword = list(df_keyword["번역"])
keyword_embeddings = []

for ele in keyword:
    keyword_embeddings.append(model.encode(ele))

In [15]:
#keyword와 바이그램 유사도 비교

bigram_result = []
keyword_result = []
cosine_result = []
for index, bigram in enumerate(bigram_embeddings): #2번 반복(총 2개 문서)
    
    b_result = ""
    k_result = ""
    c_result = ""
    
    for i in range(len(bigram)): # 20번 반복(top 20)
        
        for j in range(len(keyword_embeddings)): #102번 반복(keyword 개수)
            
            distances = cosine_similarity([bigram[i]],[keyword_embeddings[j]]) #유사도 비교
            
            if distances[0][0] > 0.7:
                
                b_result = b_result + "/" + bigram_keywords[index][i]
                k_result = k_result + "/" + keyword[j]
                c_result = c_result + "/" + str(round(float(distances),2))
                
    bigram_result.append(b_result[1:])
    keyword_result.append(k_result[1:])
    cosine_result.append(c_result[1:])

In [16]:
print(bigram_result)
print(keyword_result)
print(cosine_result)

['beers fuel/beers fuel/beers fuel/beers fuel/quarter indexation/quarter indexation/atogovaualcoholexciserates atogovaufuelexciserates/spirits beers', 'duty electric/duty electric/duty electric/hydrogen fuelcell/threshold free/threshold free/hybrid vehicles/vehicles hydrogen/car tax']
['LNG (liquid natural gas)/propane/beverage/alcohol/nickel/gourd/gourd/alcohol', 'propane/copper/machinery/LNG (liquid natural gas)/nonmetalic minerals/Solid wood/car/car/car']
['0.7/0.72/0.73/0.78/0.7/0.71/0.71/0.77', '0.76/0.71/0.7/0.74/0.72/0.74/0.72/0.7/0.76']


In [17]:
df_check = pd.DataFrame()

# for i in range(len(df)):
for i in range(2):  
    bigram = pd.Series(bigram_result[i])
    keyword = pd.Series(keyword_result[i])
    distance = pd.Series()
    if i == 0:
        df_check['bigram'] = bigram
        df_check['keyword'] = keyword
    else:
        df_check['bigram'].append(bigram)
        df_check['keyword'].append(keyword)

df_check

C:\Users\water\AppData\Local\Temp\ipykernel_18904\3130539640.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  distance = pd.Series()
C:\Users\water\AppData\Local\Temp\ipykernel_18904\3130539640.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  distance = pd.Series()
C:\Users\water\AppData\Local\Temp\ipykernel_18904\3130539640.py:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_check['bigram'].append(bigram)
C:\Users\water\AppData\Local\Temp\ipykernel_18904\3130539640.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_check['keyword'].append(keyword)


,bigram,keyword
0,beers fuel/beers fuel/beers fuel/beers fuel/qu...,LNG (liquid natural gas)/propane/beverage/alco...
